In [1]:
import numpy as np
import pandas as pd
import pickle
from colvars import Colvars
from tqdm import tqdm
from periodic_kdtree import PeriodicCKDTree

fibro = pickle.load(open("fibronectin_system.pkl",'rb'))

xbound = 2*fibro.fibronectins.side_lengthx
ybound = 2*fibro.fibronectins.side_lengthy
zbound = 2*fibro.fibronectins.side_lengthz
bounds = np.array([xbound, ybound, zbound])
# the box is defined from -side_length to +side_length
# so we need to multiply by 2

# atom types of the interacting patches
interacting_atom_types = [2,3,8,9]

df = pd.read_csv('fibro.xyz',skiprows=2,delimiter=' ',names=['type','x','y','z'])

In [3]:
# herein a dataframe  with the coordinates of the atoms and the index
# of the corresponding fibronectin monomer, please note that this is different 
# from the molecular index

fibronectin_ends = sorted(list(set(fibro.fibronectins.monomer_chunks)))
fibronectin_length = fibronectin_ends[1] - fibronectin_ends[0]
fibronectin_indices = np.array([ i//(fibronectin_length) for i in range(df.shape[0])])
fibro_array = fibronectin_indices
fibronectin_indices = pd.DataFrame({'fibronectin_indices': fibronectin_indices})
df = df.join(fibronectin_indices)


In [5]:
atomic_indices = [ i for i in range(df.shape[0]) ]
atomic_indices = pd.DataFrame({'atomic_indices': atomic_indices})
df = df.join(atomic_indices)

In [7]:
atom_to_monomer = { df['atomic_indices'][i]:df['fibronectin_indices'][i] for i in range(df.shape[0])}

In [9]:
# this filters just the atoms in the interactin patches
df = df[df['type'].isin(interacting_atom_types)]

In [11]:
coordinates = np.array([df['x'].values,df['y'].values,df['z'].values])

In [14]:
data = coordinates.T
T = PeriodicCKDTree(bounds,data)

In [17]:
class InteractionMatrixBuilder:
    
    """This class builds the interaction matrix. In this, Mij is 1 if two monomers interact and
    0 if they don't."""

    def __init__(self):
        
        self.cutoff = 1.3*1.12
        no_monomers = df[-1:]['fibronectin_indices'].values[0]+1
        self.matrix = [[0]*no_monomers for _ in range(no_monomers)]    
        
    def get_interactions(self):
        
        for i in tqdm(range(len(data))):
            distances, neighbour_indices = T.query(data[i], k=5)
            for j in range(len(neighbour_indices)):
                if distances[j] < self.cutoff and atom_to_monomer[int(df.iloc[i].atomic_indices)] != atom_to_monomer[int(df.iloc[neighbour_indices[j]].atomic_indices)]:
                    #print("interaction found")
                    #print(i,j)
                    #print(df.iloc[i].atomic_indices,df.iloc[j].atomic_indices)
                    u = atom_to_monomer[int(df.iloc[i].atomic_indices)]
                    v = atom_to_monomer[int(df.iloc[neighbour_indices[j]].atomic_indices)]
                    self.matrix[u][v],self.matrix[v][u] = 1,1  

In [18]:
interaction_matrix = InteractionMatrixBuilder()
interaction_matrix.get_interactions()

100%|██████████| 17500/17500 [00:31<00:00, 555.53it/s]
